### Imports

In [1]:
import pickle as pkl
import lxml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import importlib
import time
import traceback

from tensorflow.keras.datasets import imdb
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Conv1D, Flatten, MaxPooling1D,\
                        AveragePooling1D, Concatenate, LeakyReLU, Embedding,\
                        GlobalMaxPooling1D,GlobalAveragePooling1D,GaussianNoise,BatchNormalization,Add,ZeroPadding1D
from tensorflow.keras.initializers import glorot_uniform


importlib.reload(tensorflow)

<module 'tensorflow' from '/miniconda/lib/python3.6/site-packages/tensorflow/__init__.py'>

In [2]:
tqdm.pandas()

In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()

InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 11996954624

### Helper Functions / Classes

In [4]:
from IPython.display import clear_output
class PlotLosses(tensorflow.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()
        
        plt.plot(range(len(global_history)), global_history, label = 'Accuracy - Global')
        plt.plot(range(len(global_val_history)), global_val_history, label = 'Accuracy - Global - Va;')
        plt.legend()
        plt.show()
        
plot_losses = PlotLosses()

### Data

In [5]:
chunk_size = 100

drugbank_reader = pd.read_csv('data/intermediate/interactions_preprocessed.csv', chunksize = chunk_size, iterator = True, converters={"target_gene_encoded_padded": \
                                                                                     lambda x: np.array(x.strip("[]").replace("'","").split(", ")),
                                                                                    "drug_fingerprint_encoded_padded": \
                                                                                    lambda x: np.array(x.strip("[]").replace("'","").split(", "))})


print ("Reading data - chunk size : " + str(chunk_size) )
start = time.time()
drugbank = drugbank_reader.__next__()
end = time.time()
print ("Data read complete : " + str(end - start) + "s")



Reading data - chunk size : 100
Data read complete : 0.8506441116333008s


In [3]:
compressed_db = drugbank[['drug_id', 'gene_id']]

In [6]:
drug_dict = {}
gene_dict = {}

for drugbank in tqdm(drugbank_reader):
    
    compressed_db = drugbank[['drug_id', 'gene_id']]
    
    for i in range(len(compressed_db)):
        
        try:
            drug = compressed_db['drug_id'].values[i]
            gene = compressed_db['gene_id'].values[i]

            try:
                drug_dict[drug].add(gene)
            except: 
                drug_dict[drug] = set()
                drug_dict[drug].add(gene)

            try:
                gene_dict[gene].add(drug)
            except: 
                gene_dict[gene] = set()
                gene_dict[gene].add(drug)
        
        except Exception as e: 
            print ("Exception : " + str(e))
            traceback.print_exc()
            continue

In [7]:
print ("Number Of Unique Drugs : " + str(len(drug_dict.keys())))
print ("Number Of Unique Genes : " + str(len(gene_dict.keys())))

Number Of Unique Drugs : 6215
Number Of Unique Genes : 3774


In [8]:
drug_to_gene_count = np.zeros((len(drug_dict.keys()), 2))
gene_to_drug_count = np.zeros((len(gene_dict.keys()), 2))

In [9]:
drug_keys = list(drug_dict.keys())
gene_keys = list(gene_dict.keys())

for d in range(len(drug_keys)):
    drug_to_gene_count[d][0] = d
    drug_to_gene_count[d][1] = len(drug_dict[drug_keys[d]])
drug_to_gene_count = pd.DataFrame(drug_to_gene_count)
drug_to_gene_count.columns = ['Drug Index', 'Number Of Genes']
    
for g in range(len(gene_keys)):
    gene_to_drug_count[g][0] = g
    gene_to_drug_count[g][1] = len(gene_dict[gene_keys[g]])
gene_to_drug_count = pd.DataFrame(gene_to_drug_count)
gene_to_drug_count.columns = ['Gene Index', 'Number Of Drugs']

In [10]:
percentile_list = [1, 25, 50, 75, 99]
print ("Percentile Split Of Drug -> Gene Interaction")
for p in percentile_list: 
    print ("p" + str(p) + " : " + str(np.percentile(drug_to_gene_count['Number Of Genes'], p)) + " Genes")
print ("") 
print ("Percentile Split Of Gene -> Drug Interaction")
for p in percentile_list: 
    print ("p" + str(p) + " : " + str(np.percentile(gene_to_drug_count['Number Of Drugs'], p)) + " Drugs")

Percentile Split Of Drug -> Gene Interaction
p1 : 1.0 Genes
p25 : 2.0 Genes
p50 : 6.0 Genes
p75 : 12.0 Genes
p99 : 160.0 Genes

Percentile Split Of Gene -> Drug Interaction
p1 : 1.0 Drugs
p25 : 3.0 Drugs
p50 : 8.0 Drugs
p75 : 19.0 Drugs
p99 : 290.39999999999964 Drugs


#### Build Lists Of Similar Genes

<b>Method 1 : </b>

Randomly Sample Genes To Build Lists Of Similar Genes

This will contain genes overlapping multiple drugs. We could run this multiple times and produce multiple
datasets to train the network on.

In [156]:
number_of_genes = len(gene_dict.keys())
list_of_genes = []
genes_considered = set()

for i in range(1000):
    
    # Select A Random Gene Not Yet Selected
    count = 0
    break_flag = 0
    while gene in genes_considered:
        gene = gene_keys[np.random.randint(number_of_genes)]
        count = count + 1 
        # All Genes Have Been Considered
        if count == len(gene_dict.keys()):
            break_flag = 1
            break
    
    if break_flag == 1: 
        break

    similar_genes = []
    
    # For Each Drug That Works On This Gene
    for drug in gene_dict[gene]:
        
        # Get Other Genes Affected By Drug
        for other_gene in drug_dict[drug]:
            
            # Check If This Gene Already Belongs To Another Group
            if other_gene not in genes_considered:
                genes_considered.add(other_gene)
                similar_genes.append(other_gene)
            else: 
                continue
    
    list_of_genes.append(similar_genes)         


gene_data_distribution = []
gene_train_set = []
min_threshold = 2
for l in list_of_genes: 
    if len(l) > 2:
        gene_train_set.append(l)
        gene_data_distribution.append(len(l))
        
print ("Number Of Similar Gene Lists With Length > " + str(min_threshold) + " : " + str(len(gene_train_set)))
for p in percentile_list: 
    print ("p" + str(p) + " : " + str(np.percentile(gene_data_distribution, p)) + " Similar Genes")
print ("")    

Number Of Similar Gene Lists With Length > 2 : 81
p1 : 3.0 Similar Genes
p25 : 3.0 Similar Genes
p50 : 5.0 Similar Genes
p75 : 13.0 Similar Genes
p99 : 694.6000000000009 Similar Genes



<b>Method 2 : </b>

Pick out genes which are affected only by one drug. Then pick out other genes which are also affected 
by the same drug. 

In doing so, we ensure there is no overlap in similar genes across drugs.

In [163]:
list_of_genes = []
genes_considered = set()
single_drug_gene = []

# Find All Genes That Are Affected Only By ONE Drug 
for gene in gene_dict.keys():
    if len(gene_dict[gene]) == 1:
        single_drug_gene.append(gene)

# For Each Gene
for gene in single_drug_gene:
    
    # Skip Gene If It's Already Part Of A Similar Set
    if gene not in genes_considered: 
        
        genes_considered.add(gene)
        similar_genes = []

        # Find The Drug That Affects It
        drug = list(gene_dict[gene])[0]


        # Find Other Genes This Drug Affects And Add It To Similar Genes
        for other_gene in drug_dict[drug]:
            similar_genes.append(other_gene)
            genes_considered.add(other_gene)

        list_of_genes.append(similar_genes)
    
    else: 
        continue

gene_data_distribution = []
gene_train_set = []
min_threshold = 2
for l in list_of_genes: 
    if len(l) > min_threshold:
        gene_train_set.append(l)
        gene_data_distribution.append(len(l))
        
print ("Number Of Similar Gene Lists With Length > " + str(min_threshold) + " : " + str(len(gene_train_set)))
for p in percentile_list: 
    print ("p" + str(p) + " : " + str(np.percentile(gene_data_distribution, p)) + " Similar Genes")
print ("")    

Number Of Similar Gene Lists With Length > 2 : 116
p1 : 3.0 Similar Genes
p25 : 7.0 Similar Genes
p50 : 14.0 Similar Genes
p75 : 37.0 Similar Genes
p99 : 153.09999999999997 Similar Genes



#### Build List Of Similar Drugs

Pick out drugs which work only on one gene. Then pick out other drugs which also work on the same gene.

In doing so, we ensure there is no overlap in similar drugs across genes.

In [162]:
list_of_drugs = []
drugs_considered = set()
single_gene_drug = []

# Find All Drugs That Work Only On ONE Gene
for drug in drug_dict.keys():
    if len(drug_dict[drug]) == 1:
        single_gene_drug.append(drug)
        
# For Each Drug
for drug in single_gene_drug:
    
    # Skip Drug If It's Already Part Of A Similar Set
    if drug not in drugs_considered: 
        
        drugs_considered.add(drug)
        similar_drugs = []

        # Find The Gene That The Drug Works On
        gene = list(drug_dict[drug])[0]


        # Find Other Drugs That Work On This Gene
        for other_drug in gene_dict[gene]:
            similar_drugs.append(other_drug)
            drugs_considered.add(other_drug)

        list_of_drugs.append(similar_drugs)
    
    else: 
        continue
        
drug_data_distribution = []
drug_train_set = []
min_threshold = 2
for l in list_of_drugs: 
    if len(l) > min_threshold:
        drug_train_set.append(l)
        drug_data_distribution.append(len(l))
        
print ("Number Of Similar Drug Lists With Lenght > " + str(min_threshold) + " : " + str(len(drug_train_set)))
for p in percentile_list: 
    print ("p" + str(p) + " : " + str(np.percentile(drug_data_distribution, p)) + " Similar Drugs")
print ("")   

Number Of Similar Drug Lists With Lenght > 2 : 291
p1 : 3.0 Similar Drugs
p25 : 7.0 Similar Drugs
p50 : 14.0 Similar Drugs
p75 : 27.5 Similar Drugs
p99 : 153.30000000000007 Similar Drugs

